# Julia is fast

In [1]:
function matmul( A::Array{Float64,2}, B::Array{Float64,2})

    m, n  = size(A)
    p, q  = size(B)
    @assert n == p
    C = zeros(Float64,(m,q))
    for i = 1:m
        for j = 1:q
            C[i,j] = sum( A[i,:] .* B[:,j])
        end
    end
    C
end

matmul (generic function with 1 method)

In [2]:
@show A = rand(Float64,(4,5))
@show B = rand(Float64,(5,4))

A = rand(Float64, (4, 5)) = [0.241611 0.417429 0.228354 0.477761 0.953438; 0.612531 0.758927 0.21712 0.255127 0.617772; 0.568903 0.470955 0.650749 0.78061 0.921628; 0.823411 0.0580882 0.941221 0.266059 0.615409]
B = rand(Float64, (5, 4)) = [0.160605 0.194814 0.432959 0.508427; 0.379324 0.594297 0.820769 0.585028; 0.206791 0.496093 0.615393 0.0512913; 0.565484 0.0244044 0.959953 0.532796; 0.170678 0.661504 0.327258 0.139451]


5×4 Array{Float64,2}:
 0.160605  0.194814   0.432959  0.508427 
 0.379324  0.594297   0.820769  0.585028 
 0.206791  0.496093   0.615393  0.0512913
 0.565484  0.0244044  0.959953  0.532796 
 0.170678  0.661504   0.327258  0.139451 

In [3]:
matmul(A,B) == A * B

true

In [4]:
A * B

4×4 Array{Float64,2}:
 0.677263  1.05079  1.3584   0.766269
 0.680863  1.09295  1.4688   0.988637
 1.00331   1.34226  2.08428  1.14257 
 0.604403  1.07546  1.4402   0.728479

In [8]:
@code_llvm matmul(A,B)


; Function matmul
; Location: In[1]:3
define nonnull %jl_value_t addrspace(10)* @japi1_matmul_37047(%jl_value_t addrspace(10)*, %jl_value_t addrspace(10)**, i32) #0 {
top:
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 6
  %3 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %3, i8 0, i32 48, i32 0, i1 false)
  %4 = alloca %jl_value_t addrspace(10)**, align 8
  store volatile %jl_value_t addrspace(10)** %1, %jl_value_t addrspace(10)*** %4, align 8
  %5 = alloca { { i64 } }, align 8
  %6 = alloca { i64, { { i64 } } }, align 8
  %7 = alloca { { i64 } }, align 8
  %8 = alloca { { { i64 } }, i64 }, align 8
  %9 = alloca [1 x { i64 }], align 8
  %10 = alloca [1 x { i64 }], align 8
  %11 = call %jl_value_t*** inttoptr (i64 4536557552 to %jl_value_t*** ()*)() #5
  %12 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %13 = bitcast %jl_value_t addrspace(10)** %12 to i64*
  store i64 8, i64* %13
  %14 = gete

In [1]:
a = rand(10^7)

10000000-element Array{Float64,1}:
 0.06912063409191571 
 0.6605555484137551  
 0.7418959467920563  
 0.09866981297598443 
 0.3846042995236332  
 0.4548823803226969  
 0.8657346054689619  
 0.38989284863551    
 0.3399819051286541  
 0.8493922951810273  
 0.2614651972749511  
 0.042737990127250214
 0.5206661891205255  
 ⋮                   
 0.8637683335042361  
 0.8998884487693246  
 0.6547660649215932  
 0.3723075139295797  
 0.39007897172249306 
 0.5414786100923696  
 0.49650684863517536 
 0.9336377932892619  
 0.14889720227563075 
 0.3177230443052339  
 0.7058313775112555  
 0.6169634051674571  

In [2]:
sum(a)/length(a)

0.5000879670603202

In [3]:
using BenchmarkTools, Libdl

In [4]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
a = rand(10^7)

10000000-element Array{Float64,1}:
 0.6987951608178609 
 0.19972342860567838
 0.6743865986637485 
 0.4936267005960362 
 0.4020847296684331 
 0.7999804239881785 
 0.5565583207999754 
 0.19786558737490445
 0.11525843077482323
 0.8936256284890614 
 0.9743632813264052 
 0.42033237026053616
 0.28865281913547225
 ⋮                  
 0.5016295249786886 
 0.7962582992993255 
 0.8043036926836882 
 0.25256037625339034
 0.5039177287704752 
 0.8250510920344358 
 0.6887259603146056 
 0.7992443790703772 
 0.2669179786824609 
 0.06717141232282797
 0.1998934628722231 
 0.3044968108082102 

In [6]:
c_sum(a)

4.998491923572921e6

In [8]:
b = rand(10)

10-element Array{Float64,1}:
 0.9613552369221878 
 0.2541051629450206 
 0.5779491860874129 
 0.9772257090978973 
 0.6669808293028874 
 0.9694098903806068 
 0.37675827430129716
 0.4182268617804865 
 0.8712572646725538 
 0.5271436626312807 

In [9]:
run(`gfortran -fPIC -O3 -shared sumvec.F90 -o libsumvec.so`)

Process(`gfortran -fPIC -O3 -shared sumvec.F90 -o libsumvec.so`, ProcessExited(0))

In [10]:
function sumvec(x::Vector{Float64})
    outsum = Ref(0.0)
    N = Ref(length(x))
    ccall((:sumvec_, "./libsumvec.so"), Nothing,
        (Ptr{Float64}, Ref{Int64}, Ref{Float64}),
        x, N, outsum)
    return outsum[]
end
sumvec([1.0, 2.0])

3.0

In [11]:
X = rand(10^7)

10000000-element Array{Float64,1}:
 0.7913418556781557  
 0.17971002910667355 
 0.5872893557991585  
 0.4530415659964364  
 0.9100096911726485  
 0.5413760557393317  
 0.46332971004719736 
 0.4237333685868405  
 0.3184429005866909  
 0.5139495343006764  
 0.9854312778595864  
 0.4137041184735246  
 0.1851021963791808  
 ⋮                   
 0.10441693410961173 
 0.824145547867482   
 0.8090241638723381  
 0.6463966356164685  
 0.5198541072725753  
 0.009030983487056776
 0.5028057074176842  
 0.7776178342988742  
 0.5788767808122302  
 0.25317445547426765 
 0.8826012704344803  
 0.8700314536549074  